In [1]:
import os, sys, shutil

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from utils import vasp, slurm, structure, experiment

In [2]:
WORKDIR = os.getcwd()

PATHDICT = {
    'UTILS': os.path.join(WORKDIR, 'utils'),
    'SAMPLE': os.path.join(WORKDIR, 'sample'),
    'SMP_TEMPLATE': os.path.join(WORKDIR, 'sample/in/template'),
    'SMP_INSTANCE': os.path.join(WORKDIR, 'sample/in/instance'),
    'SMP_OUTPUT': os.path.join(WORKDIR, 'sample/out'), 
}

for path in PATHDICT.values():
    assert(os.path.isdir(path))
    
WORKDIR

'/Users/shell/Documents/Projects/ac275_final'

### Create a parameter scan

In [3]:
!ls ./sample/in/template/std

INCAR    KPOINTS  POSCAR   POTCAR   batch.sh


The following script takes these files as templates and modifies their contents to create a parameter scan.

In [4]:
# create a scan on cut-off energy from 250eV to 450eV
ecut_scan = experiment.EcutScanFromTemplate(
    src_dir=os.path.join(PATHDICT['SMP_TEMPLATE'], 'std'))
ecut_scan.make(
    out_dir=os.path.join(PATHDICT['SMP_OUTPUT'], 'ecut_scan'),
    param_list=[250, 300, 350, 400, 450],
    header='monolayer_CrI3', 
    overwrite=True)

You may go to `./sample/out` directory and see the results. The output directory has been organized as:

In [5]:
!ls ./sample/out/ecut_scan

monolayer_CrI3_ecut=250 monolayer_CrI3_ecut=350 monolayer_CrI3_ecut=450
monolayer_CrI3_ecut=300 monolayer_CrI3_ecut=400


The namings are affected by the `header` parameter.

In [6]:
!ls ./sample/out/ecut_scan/monolayer_CrI3_ecut=250

INCAR    KPOINTS  POSCAR   POTCAR   batch.sh


You may go to check if the `ENCUT` parameter in `INCAR` files has been altered. In addition, the `--job-name` in the Slurm script `batch.sh` is automatically aligned with the folder name, which helps you keep track of your calculations during runtime on Slurm without confusion.

You can scan `KPOINTS`, `SAXIS` in the same way. Lattice scans are slightly different - you need to first specify a structure generator, which takes the lattice constant as input and returns a structure resembling the ones defined in `structure.py`

In [7]:
# create a lattice scan on `a` from 6.8A to 7.2A
lattice_scan = experiment.StrucScanFromTemplate(
    src_dir=os.path.join(PATHDICT['SMP_TEMPLATE'], 'std'),
    struc_gen=lambda a: structure.MonoLayerCrI3(a, vac=20.0))
lattice_scan.make(
    out_dir=os.path.join(PATHDICT['SMP_OUTPUT'], 'lattice_scan'),
    param_list=np.arange(6.8, 7.3, 0.1), 
    header='monolayer_CrI3', 
    overwrite=True)

The above script can be easily modified to scan on the vacuum thickness, in which case the structure generator could be written as "`lambda v: structure.MonoLayerCrI3(a=7.0, vac=v)`"

Note that for demonstration convenience we have set `overwrite=True`, which tells the function to first erase the potentially existing output directory. However, users are recommended to set `overwrite=False`, which is also the default, to prevent data loss. If the purpose is to update calculation settings, add `merge=True`.

### Continue calculation

In [8]:
experiment.ToolKit.make_out_dir(
    os.path.join(PATHDICT['SMP_OUTPUT'], 'single_continue'), 
    overwrite=True)
experiment.ToolKit.copy_all(
    os.path.join(PATHDICT['SMP_INSTANCE'], 'single'),
    os.path.join(PATHDICT['SMP_OUTPUT'], 'single_continue'))

!ls ./sample/out/single_continue

CHG      DOSCAR   INCAR    OUTCAR   POTCAR   WAVECAR
CHGCAR   EIGENVAL KPOINTS  PCDAT    PROCAR   XDATCAR
CONTCAR  IBZKPT   OSZICAR  POSCAR   REPORT   batch.sh


Suppose the above directory contains a VASP calculation that has been interrupted in the middle. To continue with, one should at least copy the `CONTCAR` content to the `POSCAR`. The following script is mainly aimed at doing the same thing.

In [9]:
experiment.ToolKit.continue_relaxation(
    os.path.join(PATHDICT['SMP_OUTPUT'], 'single_continue'),
    batch_update={'-n': 16, '-t': '0-00:15'},
    push_conv=True)

!ls ./sample/out/single_continue

CHG        DOSCAR     INCAR      OUTCAR     POSCAR_old REPORT     batch.sh
CHGCAR     EIGENVAL   KPOINTS    PCDAT      POTCAR     WAVECAR
CONTCAR    IBZKPT     OSZICAR    POSCAR     PROCAR     XDATCAR


The `CONTCAR` is copied to `POSCAR`. The original `CONTCAR` stays untouched. The original `POSCAR` is renamed as `POSCAR_old`. 

The `batch_update` contains the update to the `batch.sh`. The original configuration is 30 minutes on 32 CPU cores. We reduced the numbers to half as we expect that this continue of calculation will take shorter time. The `push_conv=True` is better used when you are very close to the minimum in an ionic relaxation. It switches the relaxation method to `RMM-DIIS`.

Note that this continuation happens in-place. In real practice you don't need to copy the files out as we did at the beginning of this section - we just want to keep the `./sample/in` directory untouched.

### Develop new calculations based on an old one

In [10]:
!ls ./sample/in/instance/single

CHG      DOSCAR   INCAR    OUTCAR   POTCAR   WAVECAR
CHGCAR   EIGENVAL KPOINTS  PCDAT    PROCAR   XDATCAR
CONTCAR  IBZKPT   OSZICAR  POSCAR   REPORT   batch.sh


Suppose the above directory contains the result of a collinear VASP calculation. You have the `CHGCAR` and wants to move on to a non-collinear one to scan the spin orientation. The `batch.sh` and `INCAR` need to be changed considerably. The best practice might be preparing a new set of templates and use them to replace the old ones. The following script does the replacement and the scan preparation for you.

In [11]:
add_ncl = experiment.SaxisScanFromSTD(
    src_dir=os.path.join(PATHDICT['SMP_INSTANCE'], 'single'),
    new_dir=os.path.join(PATHDICT['SMP_TEMPLATE'], 'ncl'))
add_ncl.make(
    out_dir=os.path.join(PATHDICT['SMP_OUTPUT'], 'single_ncl'),
    param_list=[(0,0,1), (1,0,0)],
    header='single',
    overwrite=True)

In [12]:
!ls ./sample/out/single_ncl

single_saxis=[0_0_1] single_saxis=[1_0_0]


In [13]:
!ls ./sample/out/single_ncl/single_saxis=[0_0_1]

CHG        DOSCAR     INCAR      OUTCAR     POSCAR_old REPORT     batch.sh
CHGCAR     EIGENVAL   KPOINTS    PCDAT      POTCAR     WAVECAR
CONTCAR    IBZKPT     OSZICAR    POSCAR     PROCAR     XDATCAR


You may go to check whether the `batch.sh` and the `INCAR` have been replaced. Note that this script also automatically did the `CONTCAR`-`POSCAR` trick to make sure you use the latest atom positions.

### Expand the dimensions of a whole set of old scans

In [14]:
!ls ./sample/in/instance/set

monolayer_CrI3_ecut=250 monolayer_CrI3_ecut=350 submit_exhaustive.py
monolayer_CrI3_ecut=300 monolayer_CrI3_ecut=400


In [15]:
!ls ./sample/in/instance/set/monolayer_CrI3_ecut=250

CHG      DOSCAR   INCAR    OUTCAR   POTCAR   WAVECAR
CHGCAR   EIGENVAL KPOINTS  PCDAT    PROCAR   XDATCAR
CONTCAR  IBZKPT   OSZICAR  POSCAR   REPORT   batch.sh


Above we have a whole set of cut-off energy scan results of collinear VASP calculation. Suppose that you want to develop spin-orientation scans on them all. You basically need to perform the previous operation on each of the folders. The following script serves as an example.

In [16]:
template_dir = os.path.join(PATHDICT['SMP_TEMPLATE'], 'ncl')

oldset_dir = os.path.join(PATHDICT['SMP_INSTANCE'], 'set')
newset_dir = os.path.join(PATHDICT['SMP_OUTPUT'], 'set_ncl')
experiment.ToolKit.make_out_dir(newset_dir, overwrite=True)

for old_name in os.listdir(oldset_dir):
    old_path = os.path.join(oldset_dir, old_name)
    if not os.path.isdir(old_path):
        shutil.copy2(old_path, os.path.join(newset_dir, old_name))
    else:
        newset = experiment.SaxisScanFromSTD(
            src_dir=old_path, new_dir=template_dir)
        newset.make(
            out_dir=newset_dir,
            param_list=[(0,0,1), (1,0,0)], 
            incar_keeps=['ENCUT'],
            header=old_name,
            overwrite=False, 
            merge=True)
        
!ls ./sample/out/set_ncl

monolayer_CrI3_ecut=250_saxis=[0_0_1] monolayer_CrI3_ecut=350_saxis=[1_0_0]
monolayer_CrI3_ecut=250_saxis=[1_0_0] monolayer_CrI3_ecut=400_saxis=[0_0_1]
monolayer_CrI3_ecut=300_saxis=[0_0_1] monolayer_CrI3_ecut=400_saxis=[1_0_0]
monolayer_CrI3_ecut=300_saxis=[1_0_0] submit_exhaustive.py
monolayer_CrI3_ecut=350_saxis=[0_0_1]


Note the usage of `incar_keeps=['ENCUT']`. It keeps the cut-off energies untouched when replacing the template files. You may check the `INCAR` files to see this.

### Advanced usages

The `vasp.py` and `slurm.py` provides more hacks on file parsing and template adjustment.

We have a useful result parser, `vasp.VaspOUTCAR`, which has not yet been integrated to the higher-level workflow as listed above. This will be added to future work.